In [1]:
BASE_DIR ="C:/Users/jacob/OneDrive/Documents/GitHub/external_data/nlp4neuro/results_april30/experiment_6"


In [1]:
import os ,glob ,tifffile ,numpy as np ,matplotlib .pyplot as plt 

fish =11 
img_dir =os .path .join (os .path .dirname (BASE_DIR ),os .pardir ,f"fish{fish}_images")
print (img_dir )
plane0_file =glob .glob (os .path .join (img_dir ,"plane_0.*"))[0 ]

im =tifffile .imread (plane0_file ).astype (float )
im =(im -im .min ())/(im .max ()-im .min ())
im *=1.5 
im =np .clip (im ,0 ,1 )

fig ,ax =plt .subplots (figsize =(6 ,6 ),dpi =300 )
ax .imshow (im ,cmap ="viridis")

bar_len_px =SCALE_BAR_UM /UM_PER_PIXEL 
x0 ,y0 =0.02 ,0.95 

xdata0 ,ydata0 =ax .transAxes .transform ((x0 ,1 -y0 ))
xdata0 ,ydata0 =ax .transData .inverted ().transform ((xdata0 ,ydata0 ))
ax .hlines (y =ydata0 ,xmin =xdata0 ,xmax =xdata0 +bar_len_px ,
colors ="white",linewidth =3 ,transform =ax .transData ,clip_on =False )
ax .text (xdata0 +bar_len_px /2 ,ydata0 -bar_len_px *0.3 ,
f"{SCALE_BAR_UM} µm",color ="white",ha ="center",va ="top",fontsize =12 )

ax .text (0.01 ,0.99 ,"GCaMP6s",transform =ax .transAxes ,
va ="top",ha ="left",color ="white",weight ="bold",fontsize =16 )
ax .axis ("off")
out =os .path .join (BASE_DIR ,f"fish{fish}_calcium.pdf")
fig .savefig (out ,bbox_inches ="tight",transparent =True )
plt .close (fig )
print ("saved ",out )


NameError: name 'BASE_DIR' is not defined

In [2]:
import os ,glob ,ast ,numpy as np ,pandas as pd ,matplotlib .pyplot as plt ,tifffile 

fish =11 
TOP_K =30 
DOT_MIN ,DOT_MAX =50 ,300 

def bright_plane0 (f ,boost =1.5 ):
    img_dir =os .path .join (os .path .dirname (BASE_DIR ),os .pardir ,f"fish{f}_images")
    plane0 =glob .glob (os .path .join (img_dir ,"plane_0.*"))[0 ]
    im =tifffile .imread (plane0 ).astype (float )
    im =(im -im .min ())/(im .max ()-im .min ())
    return np .clip (im *boost ,0 ,1 )

def load_saliency (f ):

    root =os .path .join (BASE_DIR ,f"fish{f}")
    print (root )
    vecs =[
    np .load (os .path .join (r ,"importance.npy"))
    for r ,_ ,fs in os .walk (root )if "importance.npy"in fs 
    ]
    if not vecs :
        raise FileNotFoundError (
        f"No importance.npy files under {root}. "
        "Check the folder structure or BASE_DIR."
        )
    return np .vstack (vecs ).mean (0 )

def plane0_coords (f ):
    """Return (N×2) array of x-y coordinates + matching indices for plane_0."""
    h5 =os .path .join (os .path .dirname (BASE_DIR ),os .pardir ,
    f"fish{f}_images","functional_types_df.h5")
    h5 =os .path .normpath (h5 )
    df =pd .read_hdf (h5 )
    df0 =df [df .plane =="plane_0"]

    coords =np .vstack (
    df0 .neur_coords .apply (
    lambda v :ast .literal_eval (v )if isinstance (v ,str )else v 
    )
    )

    return coords ,df0 .index .values .astype (int )

sal =load_saliency (fish )
coords ,idx =plane0_coords (fish )
sal0 =sal [idx ]

top =np .argsort (sal0 )[-TOP_K :][::-1 ]
arr =sal0 [top ]
sizes =(arr -arr .min ())/(np .ptp (arr )+1e-9 )
sizes =sizes *(DOT_MAX -DOT_MIN )+DOT_MIN 

bg =bright_plane0 (fish )
fig ,ax =plt .subplots (figsize =(6 ,6 ))
ax .imshow (bg ,cmap ="gray",vmin =0 ,vmax =1 )
ax .scatter (coords [top ,0 ],coords [top ,1 ],
s =sizes ,c ="gray",edgecolors ="white",alpha =.8 )

ax .text (0.01 ,0.99 ,f"Fish {fish}",transform =ax .transAxes ,
va ="top",ha ="left",color ="white",weight ="bold",fontsize =16 )

SCALE_BAR_UM =50 
UM_PER_PIXEL =0.6 
bar_len_px =SCALE_BAR_UM /UM_PER_PIXEL 
x0 ,y0 =0.02 ,0.95 

xdata0 ,ydata0 =ax .transAxes .transform ((x0 ,1 -y0 ))
xdata0 ,ydata0 =ax .transData .inverted ().transform ((xdata0 ,ydata0 ))
ax .hlines (y =ydata0 ,xmin =xdata0 ,xmax =xdata0 +bar_len_px ,
colors ="white",linewidth =3 ,transform =ax .transData ,clip_on =False )
ax .text (xdata0 +bar_len_px /2 ,ydata0 -bar_len_px *0.3 ,
f"{SCALE_BAR_UM} µm",color ="white",ha ="center",va ="top",fontsize =12 )

ax .axis ("off")
fig .patch .set_alpha (0 )
ax .patch .set_alpha (0 )

out =os .path .join (BASE_DIR ,f"fish{fish}_overlay.pdf")
fig .savefig (out ,bbox_inches ="tight",transparent =True )
plt .close (fig )
print ("saved ",out )


NameError: name 'BASE_DIR' is not defined

In [3]:
import os ,glob ,ast ,numpy as np ,pandas as pd ,matplotlib .pyplot as plt ,tifffile 

fish =11 
COLOR_MAP ={"Pt":"orange","Hb":"green","Other":"purple"}

def bright_plane0 (f ,boost =1.5 ):
    img_dir =os .path .normpath (os .path .join (os .path .dirname (BASE_DIR ),os .pardir ,
    f"fish{f}_images"))
    plane0 =glob .glob (os .path .join (img_dir ,"plane_0.*"))[0 ]
    im =tifffile .imread (plane0 ).astype (float )
    im =(im -im .min ())/(im .max ()-im .min ())
    return np .clip (im *boost ,0 ,1 )

def plane0_table (f ):
    h5 =os .path .normpath (os .path .join (os .path .dirname (BASE_DIR ),os .pardir ,
    f"fish{f}_images","functional_types_df.h5"))
    df =pd .read_hdf (h5 )
    return df [df .plane =="plane_0"]

df0 =plane0_table (fish ).copy ()
df0 ["coords"]=df0 .neur_coords .apply (
lambda v :ast .literal_eval (v )if isinstance (v ,str )else v 
)
df0 ["grp"]=df0 .region .fillna ("unknown").apply (
lambda r :r if r in ("Pt","Hb")else "Other"
)

bg =bright_plane0 (fish )
fig ,ax =plt .subplots (figsize =(6 ,6 ),dpi =300 )
ax .imshow (bg ,cmap ="gray",vmin =0 ,vmax =1 )

for group ,color in COLOR_MAP .items ():
    coords_list =df0 .loc [df0 .grp ==group ,"coords"].tolist ()
    if not coords_list :
        continue 
    pts =np .vstack (coords_list )
    ax .scatter (pts [:,0 ],pts [:,1 ],
    s =40 ,c =color ,edgecolors ="white",alpha =.9 ,
    label =group )

ax .legend (loc ="lower right")
ax .axis ("off")

fig .patch .set_alpha (0 );ax .patch .set_alpha (0 )

out =os .path .join (BASE_DIR ,f"fish{fish}_clusters_overlay.pdf")
fig .savefig (out ,bbox_inches ="tight",transparent =True )
plt .close (fig )
print ("saved ",out )


NameError: name 'BASE_DIR' is not defined

In [ ]:
import os ,numpy as np ,pandas as pd ,matplotlib .pyplot as plt 
from scipy .stats import sem ,ttest_rel 

fish_list =[11 ,12 ,13 ]
GROUPS =["Pt","Hb","Other"]
COLORS ={"Pt":"orange","Hb":"green","Other":"purple"}

def load_saliency (f ):

    root =os .path .join (BASE_DIR ,f"fish{f}")
    print (root )
    vecs =[
    np .load (os .path .join (r ,"importance.npy"))
    for r ,_ ,fs in os .walk (root )if "importance.npy"in fs 
    ]
    if not vecs :
        raise FileNotFoundError (
        f"No importance.npy files under {root}. "
        "Check the folder structure or BASE_DIR."
        )
    return np .vstack (vecs ).mean (0 )

def plane0_table (f ):
    h5 =os .path .normpath (os .path .join (os .path .dirname (BASE_DIR ),os .pardir ,
    f"fish{f}_images","functional_types_df.h5"))
    df =pd .read_hdf (h5 )
    return df [df .plane =="plane_0"]

group_sals =[]
for f in fish_list :
    sal =load_saliency (f )
    df =plane0_table (f ).copy ()
    df ["grp"]=df .region .fillna ("unknown").apply (
    lambda r :r if r in ("Pt","Hb")else "Other"
    )
    vals =sal [df .index .astype (int )]
    s =pd .Series (vals ,index =df .grp ).groupby (level =0 ).mean ()
    group_sals .append (s )

df_groups =pd .concat (group_sals ,axis =1 ).reindex (GROUPS )
means =df_groups .mean (axis =1 )
errs =df_groups .apply (sem ,axis =1 )

x =np .arange (len (GROUPS ))
fig ,ax =plt .subplots (figsize =(5 ,2 ),dpi =300 )
ax .bar (x ,means ,yerr =errs ,capsize =5 ,
color =[COLORS [g ]for g in GROUPS ])
ax .set_xticks (x )
ax .set_ylim (0.00045 ,0.00095 )
ax .set_xticklabels (GROUPS )
ax .set_ylabel ("Mean saliency")

def draw_bracket (ax ,x1 ,x2 ,y ,h ,text ):
    ax .plot ([x1 ,x1 ,x2 ,x2 ],[y ,y +h ,y +h ,y ],
    lw =1.5 ,c ="black")
    ax .text ((x1 +x2 )/2 ,y +h +0.00001 ,text ,
    ha ="center",va ="bottom",fontsize =12 )

print ("Paired t-tests:")
offset =errs .max ()*0.1 
h =errs .max ()*0.1 
pairs =[(0 ,1 ),(0 ,2 ),(1 ,2 )]
for i ,j in pairs :
    grp_i ,grp_j =GROUPS [i ],GROUPS [j ]
    vals_i ,vals_j =df_groups .loc [grp_i ],df_groups .loc [grp_j ]
    t_stat ,p_val =ttest_rel (vals_i ,vals_j ,nan_policy ="omit")
    print (f"{grp_i} vs {grp_j}: p = {p_val:.4f}")
    if p_val <0.2 :
        star ="***"if p_val <0.001 else "**"if p_val <0.01 else f"p={round(p_val,1)}"
        y =max (means [i ]+errs [i ],means [j ]+errs [j ])+offset 
        draw_bracket (ax ,i ,j ,y ,h ,star )
        offset +=h *2.8 

fig .tight_layout ()
out =os .path .join (BASE_DIR ,"cluster_importance_barplot.pdf")
fig .savefig (out ,bbox_inches ="tight",transparent =True )
plt .close (fig )
print ("saved ",out )
